# Home Work

В этой работе будем иследовать одну из популярнейших рекомендательных моделей - **Latent Factor Model** - https://arxiv.org/pdf/1912.04754. 

Перед выполнением задания нужно убедиться, что прогоняется бейзлайн. Для этого:
1) Скачайте  файлы - **node2name.json** и **clickstream.parque** с необходимыми данными
2) Положите в репозиторий ноутбука и запустите код

В этой работе вам нужно:
1) перебрать параметры модели - edim,batch_size, lr, epoch , num_negatives -   (по **1 балу - 5 балов**) 
2) Тип OPTIMIZER_NAME - (**4 бала за 5 оптимизаторов**)
3) На основе имеющихся данных собрать лучшую модель (по **precision@30**) и рассчитать ее метрики (**4 бала**)
4) Попробовать другие модели (например  als - https://benfred.github.io/implicit/ , gru4rec, sasrec  ) - за sasrec на хорошем уровне сразу **10 балов**. За другие модели по **3 бала**
5) По окончанию работы в mlflow настроить графики для сравнения моделей. Можно проявить фантазию, но обязательно должно быть сравнение с бейзлайном (данный ноутбук) против других моделей
6) В mlflow залогировать последнюю версию ноутбука - необходимое условия. Либо в github, но тогда прикрепить ссылку в [mlflow](http://84.201.128.89:90/) . Эксперимент в формате - **homework-\<name\>**
7) Доп балы (**20 баллов**) тому у кого будет наибольший скор на тесте. Но ваш ноутбук должен прогонятся и быть вопроизводимым.

Суммарно за работу **20 балов**

In [1]:
import os
import random
import torch
import numpy as np

In [2]:
# задаем значение гениратора случайных чисел
RANDOM_STATE = 1

In [3]:
def set_seed(seed: int):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

## Загружаем данные

In [4]:
import json

with open('data/node2name.json', 'r') as f:
    node2name = json.load(f)

node2name = {int(k):v for k,v in node2name.items()}

In [5]:
import pandas as pd

df = pd.read_parquet('data/clickstream.parque')
df = df.head(100_000)

In [6]:
df['is_train'] = df['event_date']< df['event_date'].max() - pd.Timedelta('2 day')
df['names'] = df['node_id'].map(node2name)

In [7]:
train_cooks = df[df['is_train']]['cookie_id'].unique()
train_items = df[df['is_train']]['node_id'].unique()


df = df[(df['cookie_id'].isin(train_cooks)) & (df['node_id'].isin(train_items))]

In [8]:
user_indes, index2user_id = pd.factorize(df['cookie_id'])
df['user_index'] = user_indes

node_indes, index2node = pd.factorize(df['node_id'])
df['node_index'] = node_indes

In [9]:
df['node_index'].max()

2175

In [10]:
df_train, df_test = df[df['is_train']], df[~df['is_train']]
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


df_train.shape, df_test.shape

((96611, 7), (3333, 7))

# Определяем модель

In [11]:
import torch
from torch import nn
import random 
from tqdm.auto import tqdm

from torch.utils.data import Dataset, DataLoader


class RecDataset(Dataset):
    def __init__(self, users, items, item_per_users):
        self.users = users
        self.items = items
        self.item_per_users=item_per_users

    def __len__(self):
        return len(self.users)

    def __getitem__(self, i):
        user = self.users[i]
        return torch.tensor(user), torch.tensor(self.items[i]), self.item_per_users[user]


class LatentFactorModel(nn.Module):
    def __init__(self, edim, user_indexes, node_indexes):
        super(LatentFactorModel, self).__init__()
        self.edim = edim
        self.users = nn.Embedding(max(user_indexes) + 1, edim)
        self.items = nn.Embedding(max(node_indexes) + 1, edim)

    def forward(self, users, items):
        user_embedings = self.users(users).reshape(-1, self.edim )
        item_embedings = self.items(items)
        res = torch.einsum('be,bne->bn', user_embedings, item_embedings)
        return res 

    def pred_top_k(self, users, K=10):
        user_embedings = self.users(users).reshape(-1, self.edim )
        item_embedings = self.items.weight
        res = torch.einsum('ue,ie->ui', user_embedings, item_embedings)
        return torch.topk(res, K, dim=1)

    


def collate_fn(batch, num_negatives, num_items):
    users, target_items, users_negatives = [],[], []
    for triplets in batch:
        user, target_item, seen_item = triplets
        
        users.append(user)
        target_items.append(target_item)
        user_negatives = []
        
        while len(user_negatives)< num_negatives:
            candidate = random.randint(0, num_items)
            if candidate not in seen_item:
                user_negatives.append(candidate)
                
        users_negatives.append(user_negatives)
                
    
    positive = torch.ones(len(batch), 1)       
    negatives = torch.zeros(len(batch), num_negatives)
    labels = torch.hstack([positive, negatives])
    # print(torch.tensor(target_items))
    # print(users_negatives)
    items = torch.hstack([torch.tensor(target_items).reshape(-1, 1), torch.tensor(users_negatives)])
    return torch.hstack(users), items, labels

In [12]:
user2seen = df_train.groupby('user_index')['node_index'].agg(lambda x: list(set(x)))

# 1. Перебор параметров (с помощью оптуны)

In [13]:
# заводим mlflow
import mlflow

mlflow.set_tracking_uri('http://84.201.128.89:90/')
mlflow.set_experiment('homework-mekiselev')

<Experiment: artifact_location='mlflow-artifacts:/18', creation_time=1716048375923, experiment_id='18', last_update_time=1716048375923, lifecycle_stage='active', name='homework-mekiselev', tags={}>

In [14]:
import optuna

def calc_hitrate(df_preds, K):
    return  df_preds[df_preds['rank']<K].groupby('user_index')['relevant'].max().mean()


def calc_prec(df_preds, K):
    return  (df_preds[df_preds['rank']<K].groupby('user_index')['relevant'].mean()).mean()


def get_metrics(model: torch.nn.Module, K: int = 30):
    test_users = df_test['user_index'].unique()

    preds = model.pred_top_k(torch.tensor(test_users), K)[1].numpy()
    df_preds = pd.DataFrame({'node_index': list(preds), 'user_index': test_users, 'rank': [[j for j in range(0, K)]for i in range(len(preds))]})

    df_preds = df_preds.explode(['node_index', 'rank']).merge(
        df_test[['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
        on = ['user_index', 'node_index'],
        how='left'
    )
    df_preds['relevant'] = df_preds['relevant'].fillna(0)

    hitrate = calc_hitrate(df_preds, K)
    prec = calc_prec(df_preds, K)
    return hitrate, prec


def objective(trial):
    BATCH_SIZE = trial.suggest_int('BATCH_SIZE', 16, 100_000)
    NUM_NEGATIVES = trial.suggest_int('NUM_NEGATIVES', 1, 20)
    EDIM = trial.suggest_int('EDIM', 16, 512)
    EPOCH = trial.suggest_int('EPOCH', 3, 20)
    OPTIMIZER_NAME = 'Adam'
    LR = trial.suggest_float('LR', 1e-3, 1e1, log=True)

    train_dataset = RecDataset(df_train['user_index'].values, df_train['node_index'], user2seen)
    dataloader = DataLoader(train_dataset, shuffle=True,num_workers=0, batch_size=BATCH_SIZE,collate_fn=lambda x: collate_fn(x, NUM_NEGATIVES, max(df['node_index'].values)))

    set_seed(seed=RANDOM_STATE) # фиксируем сид (иначе не будет воспроизводимости)
    model = LatentFactorModel(EDIM, user_indes, node_indes)
    optimizer = torch.optim.Adam(model.parameters(), LR)

    for i in range(EPOCH):
        losses = []
        for i in dataloader:
            users, items, labels = i
            optimizer.zero_grad()
            logits = model(users, items)
            loss = torch.nn.functional.binary_cross_entropy_with_logits(
                logits, labels
            )
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

    hitrate, prec = get_metrics(model, K=30)

    path_to_save = f'model_weights_{trial.number}.pt'
    torch.save(model.state_dict(), path_to_save)
    # логруем результат в mlflow
    with mlflow.start_run(run_name=f'optuna-{trial.number}'):
        mlflow.log_metrics(
            {
                'hitrate-at-30': hitrate, 
                'precision-at-30': prec
            },
        )
        mlflow.log_params(
            {
            'BATCH_SIZE': BATCH_SIZE,
            'NUM_NEGATIVES': NUM_NEGATIVES,
            'EDIM': EDIM,
            'EPOCH': EPOCH,
            'OPTIMIZER_NAME': OPTIMIZER_NAME,
            'LR': EPOCH,
            }
        )
        # mlflow.log_artifact( # не работает с большими файлами
        #     path_to_save,
        #     'models'
        # )
    return prec # будем максимизировать precision

In [15]:
# запускаем оптуну
tpe_sampler = optuna.samplers.TPESampler(seed=RANDOM_STATE)
study = optuna.create_study(sampler=tpe_sampler, direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-05-25 13:09:15,592] A new study created in memory with name: no-name-14bfadf9-c4e3-42dd-8773-940b068e9590


Random seed set as 1


[I 2024-05-25 13:09:35,629] Trial 0 finished with value: 0.0013493253373313345 and parameters: {'BATCH_SIZE': 41711, 'NUM_NEGATIVES': 15, 'EDIM': 16, 'EPOCH': 8, 'LR': 0.00386387940515873}. Best is trial 0 with value: 0.0013493253373313345.


Random seed set as 1


[I 2024-05-25 13:09:54,450] Trial 1 finished with value: 0.04497751124437781 and parameters: {'BATCH_SIZE': 9248, 'NUM_NEGATIVES': 4, 'EDIM': 187, 'EPOCH': 10, 'LR': 0.14297724879798399}. Best is trial 1 with value: 0.04497751124437781.


...

[I 2024-05-25 13:50:18,312] Trial 47 finished with value: 0.041929035482258875 and parameters: {'BATCH_SIZE': 83062, 'NUM_NEGATIVES': 17, 'EDIM': 139, 'EPOCH': 9, 'LR': 2.432144064183068}. Best is trial 12 with value: 0.047426286856571714.


Random seed set as 1


[I 2024-05-25 13:50:42,542] Trial 48 finished with value: 0.03633183408295852 and parameters: {'BATCH_SIZE': 47837, 'NUM_NEGATIVES': 19, 'EDIM': 370, 'EPOCH': 5, 'LR': 3.6899910487430927}. Best is trial 12 with value: 0.047426286856571714.


Random seed set as 1


[I 2024-05-25 13:52:06,529] Trial 49 finished with value: 0.04557721139430285 and parameters: {'BATCH_SIZE': 65858, 'NUM_NEGATIVES': 18, 'EDIM': 406, 'EPOCH': 17, 'LR': 6.374981357852146}. Best is trial 12 with value: 0.047426286856571714.


In [16]:
optuna_best_params = study.best_params
optuna_best_params

{'BATCH_SIZE': 68114,
 'NUM_NEGATIVES': 18,
 'EDIM': 329,
 'EPOCH': 14,
 'LR': 5.106820625932442}

# 2. Перебираем оптимизаторы

In [17]:
def fit_model(name_trial: str, config: dict):
    BATCH_SIZE = config['BATCH_SIZE']
    NUM_NEGATIVES = config['NUM_NEGATIVES']
    EDIM = config['EDIM']
    EPOCH = config['EPOCH']
    OPTIMIZER_NAME = config['optimizer'].__name__
    LR = config['LR']

    train_dataset = RecDataset(df_train['user_index'].values, df_train['node_index'], user2seen)
    dataloader = DataLoader(train_dataset, shuffle=True,num_workers=0, batch_size=BATCH_SIZE,collate_fn=lambda x: collate_fn(x, NUM_NEGATIVES, max(df['node_index'].values)))

    set_seed(seed=RANDOM_STATE) # фиксируем сид (иначе не будет воспроизводимости)
    model = LatentFactorModel(EDIM, user_indes, node_indes)
    optimizer = config['optimizer'](model.parameters(), LR)

    for i in range(EPOCH):
        losses = []
        for i in dataloader:
            users, items, labels = i
            optimizer.zero_grad()
            logits = model(users, items)
            loss = torch.nn.functional.binary_cross_entropy_with_logits(
                logits, labels
            )
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

    hitrate, prec = get_metrics(model, K=30)

    path_to_save = f'model_weights_{name_trial}.pt'
    torch.save(model.state_dict(), path_to_save)
    # логруем результат в mlflow
    with mlflow.start_run(run_name=name_trial):
        mlflow.log_metrics(
            {
                'hitrate-at-30': hitrate, 
                'precision-at-30': prec
            },
        )
        mlflow.log_params(
            {
            'BATCH_SIZE': BATCH_SIZE,
            'NUM_NEGATIVES': NUM_NEGATIVES,
            'EDIM': EDIM,
            'EPOCH': EPOCH,
            'OPTIMIZER_NAME': OPTIMIZER_NAME,
            'LR': EPOCH,
            }
        )
        # mlflow.log_artifact( # не работает с большими файлами
        #     path_to_save,
        #     'models'
        # )
    return hitrate, prec

In [18]:
from torch.optim import AdamW, Adagrad, RMSprop, SGD, ASGD

# Adam уже есть
for optimizer in [AdamW, Adagrad, RMSprop, SGD, ASGD]:
    hitrate, prec = fit_model(
        name_trial=f'optimizer: {optimizer.__name__}',
        config={**optuna_best_params, 'optimizer': optimizer}
    )
    print(f'{optimizer.__name__}:\n\thitrate: {hitrate}\n\tprecision: {prec}', end='\n\n')

Random seed set as 1
AdamW:
	hitrate: 0.7301349325337332
	precision: 0.04532733633183408

Random seed set as 1
Adagrad:
	hitrate: 0.7226386806596702
	precision: 0.04532733633183408

Random seed set as 1
RMSprop:
	hitrate: 0.7346326836581709
	precision: 0.04672663668165917

Random seed set as 1
SGD:
	hitrate: 0.037481259370314844
	precision: 0.0012493753123438282

Random seed set as 1
ASGD:
	hitrate: 0.037481259370314844
	precision: 0.0012493753123438282



# 3. Выбор лучшей модели

Среди всех моделей лучшей оказалась модель с 12 итерации оптуны с `precision@30: 0.0474`. Ещё раз соберем её

In [19]:
BATCH_SIZE = optuna_best_params['BATCH_SIZE']
NUM_NEGATIVES = optuna_best_params['NUM_NEGATIVES']
EDIM = optuna_best_params['EDIM']
EPOCH = optuna_best_params['EPOCH']
OPTIMIZER_NAME = 'Adam'
LR = optuna_best_params['LR']

train_dataset = RecDataset(df_train['user_index'].values, df_train['node_index'], user2seen)
dataloader = DataLoader(train_dataset, shuffle=True,num_workers=0, batch_size=BATCH_SIZE,collate_fn=lambda x: collate_fn(x, NUM_NEGATIVES, max(df['node_index'].values)))

set_seed(seed=RANDOM_STATE) # фиксируем сид (иначе не будет воспроизводимости)
model = LatentFactorModel(EDIM, user_indes, node_indes)
optimizer = torch.optim.Adam(model.parameters(), LR)

for i in range(EPOCH):
    losses = []
    for i in dataloader:
        users, items, labels = i
        optimizer.zero_grad()
        logits = model(users, items)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(
            logits, labels
        )
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

hitrate, prec = get_metrics(model, K=30)

print(hitrate, prec)

Random seed set as 1
0.7421289355322339 0.047426286856571714


In [20]:
# mlflow.log_artifact( # не работает с большими файлами
#     '/Users/max/Desktop/aaa-versioning-hw/model_weights_12.pt'
# )

# 4. Пробуем другие модели

## ALS

In [21]:
from scipy.sparse import csr_matrix

N_NODES = df.loc[df['is_train'], 'node_index'].nunique()

def matrix_string(list: list):
    items_string = [0] * N_NODES
    for item in list:
        items_string[int(item)] = 1
    return items_string

train_user_item_data_list = df.loc[df['is_train'], ['user_index', 'node_index']].groupby('user_index').apply(lambda x: matrix_string(x['node_index'])).tolist()
train_user_item_data = csr_matrix(train_user_item_data_list)

/var/folders/ds/gg0lwy_j1_lg7j7yvx6vcqx00000gn/T/ipykernel_9963/1797524773.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_user_item_data_list = df.loc[df['is_train'], ['user_index', 'node_index']].groupby('user_index').apply(lambda x: matrix_string(x['node_index'])).tolist()


In [22]:
def calc_hitrate(df_preds, K):
    print(df_preds[df_preds['rank']<K].groupby('user_index')['relevant'])
    return  df_preds[df_preds['rank']<K].groupby('user_index')['relevant'].max().mean()

def calc_prec(df_preds, K):
    return  (df_preds[df_preds['rank']<K].groupby('user_index')['relevant'].mean()).mean()

In [23]:
import implicit

K = 30

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=16, iterations=20, num_threads=0, random_state=RANDOM_STATE)
# train the model on a sparse matrix of user/item/confidence weights
model.fit(train_user_item_data)

test_users = df_test['user_index'].unique()

preds = model.recommend(test_users, train_user_item_data[test_users].tocsr(), N=K)[0]
df_preds = pd.DataFrame({'node_index': list(preds), 'user_index': test_users, 'rank': [[j for j in range(0, K)]for i in range(len(preds))]})

df_preds = df_preds.explode(['node_index', 'rank']).merge(
    df_test[['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
    on = ['user_index', 'node_index'],
    how='left'
)
df_preds['relevant'] = df_preds['relevant'].fillna(0)

hitrate = calc_hitrate(df_preds, K)
prec = calc_prec(df_preds, K)

with mlflow.start_run(run_name='AlternatingLeastSquares'):
    mlflow.log_metrics(
        {
            'hitrate-at-30': hitrate, 
            'precision-at-30': prec
        },
    )
    mlflow.log_params(
        {
            'factors': model.factors,
            'iterations': model.iterations,
            'num_threads': model.num_threads,
            'random_state': model.random_state
        }
    )

print(hitrate, prec)

/opt/homebrew/Caskroom/miniconda/base/envs/base_env/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/20 [00:00<?, ?it/s]

0.24587706146926536 0.009545227386306848


## SASREC

In [24]:
# по итогу не смог разобраться в доке библиотеки :pepe-clown:

from recommenders.datasets.amazon_reviews import get_review_data
from recommenders.datasets.split_utils import filter_k_core
from recommenders.models.sasrec.model import SASREC
from recommenders.models.sasrec.ssept import SSEPT
from recommenders.models.sasrec.sampler import WarpSampler
from recommenders.models.sasrec.util import SASRecDataSet
from recommenders.utils.notebook_utils import store_metadata
from recommenders.utils.timer import Timer

In [25]:
num_epochs = 5
batch_size = 128
seed = 100  # Set None for non-deterministic result

lr = 0.001             # learning rate
maxlen = 200            # maximum sequence length for each user
num_blocks = 2         # number of transformer blocks
hidden_units = 100     # number of units in the attention calculation
num_heads = 1          # number of attention heads
dropout_rate = 0.1     # dropout rate
l2_emb = 0.0           # L2 regularization coefficient
num_neg_test = 100     # number of negative examples per positive example

In [26]:
df['is_train'] = df['event_date']< df['event_date'].max() - pd.Timedelta('2 day') # train тот же самый
df['is_val'] = (df['event_date'] >= df['event_date'].max() - pd.Timedelta('3 day')) & df['is_train']
df['is_train'] = df['is_train'] & ~df['is_val']
df['is_test'] = (~df['is_train']) & (~df['is_val'])

print(df['is_train'].sum() + df['is_val'].sum() + df['is_test'].sum() == df.shape[0])
print(df['is_train'].sum(), df['is_val'].sum(), df['is_test'].sum())

True
95084 1527 3333


In [27]:
user_items = dict(df.groupby('user_index')['node_index'].apply(list))

train_user_items = dict(df[df['is_train']].groupby('user_index')['node_index'].apply(list))
val_user_items = dict(df[df['is_val']].groupby('user_index')['node_index'].apply(list))
test_user_items = dict(df[df['is_test']].groupby('user_index')['node_index'].apply(list))

In [28]:
# initiate a dataset class 
# train_data = SASRecDataSet(filename='./train_data.csv', col_sep="\t")
train_data = SASRecDataSet(
    filename='./full_data.csv',
    col_sep='\t',
    usernum=df['user_index'].unique(),
    itemnum=df['node_index'].unique(),
    User=user_items,
    Items=set(df['node_index']),
    user_train=train_user_items,
    user_valid=val_user_items,
    user_test=test_user_items
)

# create train, validation and test splits
train_data.split()

# some statistics
num_steps = int(len(train_data.user_train) / batch_size)
cc = 0.0
for u in train_data.user_train:
    cc += len(train_data.user_train[u])
print('%g Users and %g items' % (train_data.usernum, train_data.itemnum))
print('average sequence length: %.2f' % (cc / len(train_data.user_train)))

1663 Users and 2175 items
average sequence length: 58.28


In [29]:
sampler = WarpSampler(train_data.user_train, train_data.usernum, train_data.itemnum, batch_size=batch_size, maxlen=maxlen, n_workers=3)

In [30]:
model = SASREC(item_num=train_data.itemnum,
                seq_max_len=maxlen,
                num_blocks=num_blocks,
                embedding_dim=hidden_units,
                attention_dim=hidden_units,
                attention_num_heads=num_heads,
                dropout_rate=dropout_rate,
                conv_dims = [100, 100],
                l2_reg=l2_emb,
                num_neg_test=num_neg_test
)

In [31]:
t_test = model.train(train_data, sampler, num_epochs=num_epochs, batch_size=batch_size, lr=lr, val_epoch=6)


epoch: 5, test (NDCG@30: 0.6178284833856107, HR@30: 0.9151515151515152)
